https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis

este es otro https://www.kaggle.com/code/aslanahmedov/market-basket-analysis-apriori-algorithm/notebook

About Dataset
Market Basket Analysis
Market basket analysis with Apriori algorithm

The retailer wants to target customers with suggestions on itemset that a customer is most likely to purchase .I was given dataset contains data of a retailer; the transaction data provides data around all the transactions that have happened over a period of time. Retailer will use result to grove in his industry and provide for customer suggestions on itemset, we be able increase customer engagement and improve customer experience and identify customer behavior. I will solve this problem with use Association Rules type of unsupervised learning technique that checks for the dependency of one data item on another data item.

Introduction
Association Rule is most used when you are planning to build association in different objects in a set. It works when you are planning to find frequent patterns in a transaction database. It can tell you what items do customers frequently buy together and it allows retailer to identify relationships between the items.

An Example of Association Rules
Assume there are 100 customers, 10 of them bought Computer Mouth, 9 bought Mat for Mouse and 8 bought both of them.

bought Computer Mouth => bought Mat for Mouse
support = P(Mouth & Mat) = 8/100 = 0.08
confidence = support/P(Mat for Mouse) = 0.08/0.09 = 0.89
lift = confidence/P(Computer Mouth) = 0.89/0.10 = 8.9
This just simple example. In practice, a rule needs the support of several hundred transactions, before it can be considered statistically significant, and datasets often contain thousands or millions of transactions.
Strategy
Data Import
Data Understanding and Exploration
Transformation of the data – so that is ready to be consumed by the association rules algorithm
Running association rules
Exploring the rules generated
Filtering the generated rules
Visualization of Rule
Dataset Description
File name: Assignment-1_Data

List name: retaildata

File format: . xlsx

Number of Row: 522065

Number of Attributes: 7

BillNo: 6-digit number assigned to each transaction. Nominal.
Itemname: Product name. Nominal.
Quantity: The quantities of each product per transaction. Numeric.
Date: The day and time when each transaction was generated. Numeric.
Price: Product price. Numeric.
CustomerID: 5-digit number assigned to each customer. Nominal.
Country: Name of the country where each customer resides. Nominal.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

from plotly.offline import plot, iplot, init_notebook_mode
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances

In [21]:
dataset=pd.read_csv('../data/raw/Assignment-1_Data.csv',sep=';', date_parser=True)

/tmp/ipykernel_250441/2746298970.py:1: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



In [26]:
dataset.sample(5)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
190182,553884,LUNCH BAG WOODLAND,10,19.05.2011 15:36,"1,65",12600.0,Germany
24534,538356,CHRISTMAS LIGHTS 10 SANTAS,3,10.12.2010 15:56,"8,5",14062.0,United Kingdom
364326,569659,RED APPLES CHOPPING BOARD,1,05.10.2011 13:04,"4,95",14099.0,United Kingdom
89856,544206,HEART IVORY TRELLIS SMALL,1,17.02.2011 10:33,"2,46",NaN,United Kingdom
336484,567514,GIANT 50'S CHRISTMAS CRACKER,1,20.09.2011 15:10,"2,89",16928.0,United Kingdom


In [22]:
dataset.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [27]:
#Checking for missing values
#No missing values in our data
dataset.isna().sum()

BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64

In [28]:
#Extracting date information into Day,Month,Year
def split_date(data):
    data['Date']=pd.to_datetime(data['Date'])
    data['Year']=data.Date.dt.year
    data['Month']=data.Date.dt.month
    data['Day']=data.Date.dt.day

In [29]:
#Spliting our date data into day ,month and year
split_date(dataset)
#check how now our data looks like
dataset.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,Year,Month,Day
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,"2,55",17850.0,United Kingdom,2010,1,12
1,536365,WHITE METAL LANTERN,6,2010-01-12 08:26:00,"3,39",17850.0,United Kingdom,2010,1,12
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,"2,75",17850.0,United Kingdom,2010,1,12
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,"3,39",17850.0,United Kingdom,2010,1,12
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,"3,39",17850.0,United Kingdom,2010,1,12


In [31]:
#all our data is for the  Year 2010 and 2011
dataset.Year.value_counts()

2011    480822
2010     41242
Name: Year, dtype: int64

In [33]:
#Checking the data across the months (data available for the months of JAN-DEC)
dataset.Month.value_counts()

11    71148
10    54819
9     49324
7     44898
6     44085
8     43312
5     41138
1     38260
3     37326
4     36218
2     32814
12    28722
Name: Month, dtype: int64

### Exploratory Data Analysis
#### Customers Gender Distribution

In [34]:
#get the total numner of customers in our dataset
print(dataset['CustomerID'].count())
#Get total numbe of customers 
dataset['CustomerID'].value_counts()

388023


17841.0    7847
14096.0    5111
12748.0    4596
14606.0    2700
15311.0    2379
           ... 
12346.0       1
18268.0       1
17846.0       1
16742.0       1
15940.0       1
Name: CustomerID, Length: 4297, dtype: int64